In [3]:
#Библиотеки для математических вычислений и работы с данными
import math
from numpy import Inf
import pandas as pd
import csv
#Библиотека для работы с геоданными
import geopandas as gpd

#Библиотека для работы с временем
from datetime import datetime, date, timedelta

#Библиотека для получения начальных условий из открытой базы данных spacetrack.org
import spacetrack.operators as op
from spacetrack import SpaceTrackClient

#Библиотека для построения трассировки КА
from pyorbital.orbital import Orbital

#Библиотеки для работы с шейпфайлами (shapefiles)
import shapefile
import fiona
from shapely.geometry import Point, Polygon, mapping

#Библиотеки для создания интерактивной карты и визуализации
import folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
import matplotlib.pyplot as plt

import csv

print('All imported!')

All imported!


In [12]:
papka = str(input('Введите рабочую папку с подспутниковыми точками:'))
sat = int(input('Введите номер КА в NORAD:')

Введите рабочую папку с подспутниковыми точками: C:/Users/Sirius/Desktop/Тест полосы обзора
Введите номер КА в NORAD: 56756


In [15]:
def nearest_subsatellite_point(input_shapefile_tpv, input_papka_pt): #входной файл с точками
    min_s=99999999999999999
    time=[]
    a=-1
    #Обработка данных о подспутниковых точках
    file_tpv = gpd.read_file(input_shapefile_tpv)
    df_tpv = pd.DataFrame(file_tpv)
    gdf_tpv = gpd.GeoDataFrame(df_tpv, geometry = gpd.points_from_xy(df_tpv.lat, df_tpv.lon))
    for l in range(1, df_tpv['vitok'].max()+1):
        file_pt = gpd.read_file(input_papka_pt+'/{}/vitok_{}_pol_{}.shp'.format(l, l, sat))
        df_pt = pd.DataFrame(file_pt)
        gdf_pt = gpd.GeoDataFrame(df_pt, geometry = gpd.points_from_xy(df_pt.LAT, df_pt.LON))
        for j in range(len(file_tpv)): #перебор точек в полигонах
            if l==file_tpv['vitok'][j]:
                for i in range(len(file_pt)): #перебор точек в трасс
                    if min_s>((file_tpv["lat"][j]-file_pt["LAT"][i])**2+(file_tpv["lon"][j]-file_pt["LON"][i])**2):
                        min_s=(file_tpv["lat"][j]-file_pt["LAT"][i])**2+(file_tpv["lon"][j]-file_pt["LON"][i])**2
                        a=i
                time.append(file_pt["TIME"][a])
                min_s=99999999999999999
                a=-1
    print(time)
    print(len(time))
    with open(papka + "/shooting_time_all.csv", "w", newline="") as file:  
        writer = csv.writer(file)  
        writer.writerow(['TIME'])  
        for item in time:  
            writer.writerow([item])  
    print("CSV file created: shooting_time_all.csv")

In [16]:
nearest_subsatellite_point(papka + '/shooting_points.shp', papka)

['2025-07-15 01:07:00', '2025-07-15 06:04:30', '2025-07-15 07:37:00', '2025-07-15 07:39:00', '2025-07-15 09:11:00', '2025-07-15 09:11:00', '2025-07-15 10:44:00', '2025-07-15 10:45:00', '2025-07-15 10:46:30', '2025-07-15 12:19:30', '2025-07-15 12:18:00', '2025-07-15 12:23:30', '2025-07-15 13:55:30', '2025-07-15 13:57:30', '2025-07-15 13:54:00', '2025-07-15 13:52:30', '2025-07-15 13:51:00', '2025-07-15 15:24:30', '2025-07-15 15:29:30', '2025-07-15 15:28:30', '2025-07-15 15:26:30', '2025-07-15 15:25:30', '2025-07-15 16:58:30', '2025-07-15 16:57:30', '2025-07-15 16:55:30', '2025-07-15 17:03:30', '2025-07-15 17:02:00', '2025-07-15 17:00:30', '2025-07-15 18:33:30', '2025-07-15 18:36:30', '2025-07-15 18:30:00', '2025-07-15 20:07:00', '2025-07-15 20:10:00', '2025-07-15 21:42:30', '2025-07-15 21:44:30', '2025-07-15 23:14:30', '2025-07-15 23:18:00']
37
CSV file created: shooting_time_all.csv
